In [1]:
import json
import requests
import datetime
import pandas as pd
import urllib.request
import codecs
import numpy as np
from six.moves import urllib

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [3]:
#eligibility criteria
volume_cutoff = 0.1
percent_float_cutoff = 0.1

#month of rebalance
old_month = 'july'
new_month = 'august'

In [4]:
#change call to new filepath
asset_db = pd.read_excel('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/utils/rebalance_tools/csvs\\Bletchley Rebalance '+new_month+' 2020.xlsx',\
                         sheetname='assets',index_col='Ticker')

In [5]:
#grab all assets from Messari
#deprecated as they have too many!
#url = "https://data.messari.io/api/v1/assets?with-profiles"
#response = (urllib.request.urlopen(url).read())
#response = response.decode('utf-8')
#d = json.loads(response)
#df = pd.DataFrame(d['data'])
#df = df.set_index((df['symbol']))

In [6]:
df = asset_db
#ticker mapping issues between Bletchley DB and OnChainFX
df.index = map(lambda x: x.lower(), df.index)

In [7]:
url_messari = "https://data.messari.io/api/v1/assets/btc/metrics"
response_messari = (urllib.request.urlopen(url_messari).read())
response_messari = response_messari.decode('utf-8')
d_messari = json.loads(response_messari)
df_all = pd.Series()

In [8]:
#get Messari data given a ticker
#don't actually use the category or sector data
def get_messari_data(ticker):
    url_messari = "https://data.messari.io/api/v1/assets/{}/metrics".format(ticker)
    response_messari = (urllib.request.urlopen(url_messari).read())
    response_messari = response_messari.decode('utf-8')
    d_messari = json.loads(response_messari)
    df_all = pd.Series()
    if d_messari['data']['supply']['y_2050'] == None:
        df_all['Y2050 Supply'] = (d_messari['data']['supply']['circulating'])
    else:
        df_all['Y2050 Supply'] = (d_messari['data']['supply']['y_2050'])
    df_all['Supply % Issued'] = df_all['Y2050 Supply'] / ((df_all['Y2050 Supply']))
    df_all['Price'] = (d_messari['data']['market_data']['price_usd'])
    #df_all['Est Volume'] = ((d_messari['data']['market_data']['volume_last_24_hours']) * 30)
    df_all['Marketcap'] = df_all['Price'] * df_all['Y2050 Supply']
    df_all['Current Marketcap'] = df_all['Price'] * df_all['Y2050 Supply']
    try:
        df_all['Est Volume'] = ((d_messari['data']['market_data']['volume_last_24_hours']) * 180)
    except:
        pass
    return df_all

In [9]:
url_messari = "https://data.messari.io/api/v1/assets/elf/metrics"
response_messari = (urllib.request.urlopen(url_messari).read())
response_messari = response_messari.decode('utf-8')
d_messari = json.loads(response_messari)

In [65]:
a = df.index.tolist()

In [66]:
len(a)

143

In [58]:
metrics_log['CNX']

In [109]:
#master function to get all Messari data and put in dictionary
metrics_log = dict.fromkeys(df.index)
for t in df.index[133:143]:
    try:
        print(t)
        metrics = get_messari_data(t)
        print(t)
        metrics_log[t] = metrics
        df.loc[t,('Y2050 Supply')] = metrics['Y2050 Supply']
        df.loc[t,('Available Supply')] = metrics['Y2050 Supply']
        df.loc[t,('Supply % Issued')] = metrics['Supply % Issued']
        df.loc[t,('Price')] = metrics['Price']
        df.loc[t,('Marketcap')] = metrics['Marketcap']
        df.loc[t,('Current Marketcap')] = metrics['Current Marketcap']
        try:
            df.loc[t,('Est Volume')] = metrics['Est Volume']
        except:
            pass
    except:
        pass

QKC
QKC
NULS
NULS
NEXO
NEXO
WAX
WAX
IOTX
IOTX
EDO
EDO
ELA
ELA
PAI
PAI
ATOM
ATOM
BNT
BNT


In [110]:
#t = 'IOTA'
#metrics = get_messari_data(t)
#t = 'IOT'
#print (metrics)
#metrics_log[t] = metrics
#df.loc[t,('Y2050 Supply')] = metrics['Y2050 Supply']
#df.loc[t,('Available Supply')] = metrics['Available Supply']
#df.loc[t,('Est Volume')] = metrics['Est Volume']
#df.loc[t,('Supply % Issued')] = metrics['Supply % Issued']
#df.loc[t,('Price')] = metrics['Price']
#df.loc[t,('Marketcap')] = metrics['Marketcap']
#df.loc[t,('Current Marketcap')] = metrics['Current Marketcap']
#df.loc[t,('Category')] = metrics['Category']
#df.loc[t,('Sectors')] = metrics['Sectors']
#try:
#    df.loc[t,('Est Volume')] = metrics['Est Volume']
#except:
#    pass
#try:
#    df.loc[t,('Category2')] = metrics['Category2']
#except:
#    pass
#try:
#    df.loc[t,('Sectors2')] = metrics['Sectors2']
#except:
#    pass

In [111]:
# t = 'ELF'
# metrics = get_messari_data(t)
# t = 'ELF_TEST'
# #print (metrics)
# metrics_log[t] = metrics
# metrics['Y2050 Supply']
# df.loc[t,('Y2050 Supply')] = metrics['Y2050 Supply']
# df.loc[t,('Available Supply')] = metrics['Available Supply']
# df.loc[t,('Est Volume')] = metrics['Est Volume']
# df.loc[t,('Supply % Issued')] = metrics['Supply % Issued']
# df.loc[t,('Price')] = metrics['Price']
# df.loc[t,('Marketcap')] = metrics['Marketcap']
# df.loc[t,('Current Marketcap')] = metrics['Current Marketcap']
# df.loc[t,('Category')] = metrics['Category']
# df.loc[t,('Sectors')] = metrics['Sectors']
# try:
#    df.loc[t,('Est Volume')] = metrics['Est Volume']
# except:
#    pass
# try:
#    df.loc[t,('Category2')] = metrics['Category2']
# except:
#    pass
# try:
#    df.loc[t,('Sectors2')] = metrics['Sectors2']
# except:
#    pass

In [112]:
metrics_log['BNT']

Y2050 Supply         6.914855e+07
Supply % Issued      1.000000e+00
Price                1.498309e+00
Marketcap            1.036059e+08
Current Marketcap    1.036059e+08
Est Volume           9.078317e+09
dtype: float64

In [113]:
# t = 'BNT'
# metrics = get_messari_data(t)
# metrics_log[t] = metrics
# df.loc[t,('Y2050 Supply')] = metrics['Y2050 Supply']
# df.loc[t,('Available Supply')] = metrics['Available Supply']
# #df.loc[t,('Est Volume')] = metrics['Est Volume']
# df.loc[t,('Supply % Issued')] = metrics['Supply % Issued']
# df.loc[t,('Price')] = metrics['Price']
# df.loc[t,('Marketcap')] = metrics['Marketcap']
# df.loc[t,('Current Marketcap')] = metrics['Current Marketcap']
# df.loc[t,('Category')] = metrics['Category']
# df.loc[t,('Sectors')] = metrics['Sectors']
# try:
#     df.loc[t,('Est Volume')] = metrics['Est Volume']
# except:
#     pass
# try:
#     df.loc[t,('Category2')] = metrics['Category2']
# except:
#     pass
# try:
#     df.loc[t,('Sectors2')] = metrics['Sectors2']
# except:
#     pass

In [114]:
#change Messari data to uppercase to match other sources
df.index = map(lambda x: x.upper(), df.index)

In [115]:
#df.loc['IOT'] = df.loc['IOTA']
#df.loc['VET'] = df.loc['VEN']

In [116]:
#read in other data sources
onchainfx_db = df

In [117]:
#remove duplicates
#print (len(coinmarketcap_db))
print (len(onchainfx_db))
print (len(asset_db))
#coinmarketcap_db = coinmarketcap_db[~coinmarketcap_db.index.duplicated(keep='first')]
onchainfx_db = onchainfx_db[~onchainfx_db.index.duplicated(keep='first')]
asset_db = asset_db[~asset_db.index.duplicated(keep='first')]
#print (len(coinmarketcap_db))
print (len(onchainfx_db))
print (len(asset_db))

143
142
142
142


In [118]:
#fix any ticker mapping issues - rank date use these
#onchainfx_db = onchainfx_db.rename({'HSR' : 'HC'})
#onchainfx_db = onchainfx_db.rename({'HSR' : 'HC'})
onchainfx_db = onchainfx_db.rename({'IOTA' : 'MIOTA'})
onchainfx_db = onchainfx_db.rename({'IOT' : 'MIOTA'})
asset_db = asset_db.rename({'IOTA' : 'MIOTA'})

In [119]:
# #fix any ticker mapping issues - rebalance day use these
# onchainfx_db = onchainfx_db.rename({'HSR' : 'HC'})
# onchainfx_db = onchainfx_db.rename({'HSR' : 'HC'})
# onchainfx_db = onchainfx_db.rename({'IOTA' : 'IOT'})
# asset_db = asset_db.rename({'IOTA' : 'IOT'})

In [120]:
#onchainfx_db['Marketcap'] = np.where(onchainfx_db['Marketcap'].isnull() == True,onchainfx_db['Current Marketcap'],onchainfx_db['Marketcap'])

In [121]:
onchainfx_db.loc['BNT']

Coin                      Bancor
Class                          3
Sector                       NaN
Quality Exchange?              1
ERC20                          1
ICO ok?                        1
Eligible                       1
Y2050 Supply         6.91486e+07
Available Supply     6.91486e+07
Supply % Issued                1
Price                    1.49831
Marketcap            1.03606e+08
Current Marketcap    1.03606e+08
Est Volume           9.07832e+09
Name: BNT, dtype: object

In [122]:
#read in previous month Bletchley Data
list_indexes = ['10' , '20' , '40' , 'Total' , 'ETH' , 'Currency' , 'Platform' , 'Application']
indexes_old = dict.fromkeys(list_indexes)
indexes_new_tickers = dict.fromkeys(list_indexes)
indexes_new_file = dict.fromkeys(list_indexes)
for k in indexes_old:
    if (k == 'Currency' or k == 'Platform' or k == 'Application'):
        indexes_old[k] = pd.DataFrame()
        indexes_new_tickers[k] = pd.DataFrame()
        indexes_new_file[k] = pd.DataFrame(columns=['Float'])
        indexes_old[k] = pd.read_csv('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/index-data/src/main/resources/business_rules/'+k+'_'+old_month+'_rebal.csv' , index_col=[0] , names=[k,'Adj Float'])
    else:
        indexes_old[k] = pd.DataFrame()
        indexes_new_tickers[k] = pd.DataFrame()
        indexes_new_file[k] = pd.DataFrame(columns=['Float'])
        indexes_old[k] = pd.read_csv('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/index-data/src/main/resources/business_rules/'+k+'_'+old_month+'_rebal.csv' , index_col=[0] , names=[k,'Float' , 'Adj Float'])

In [123]:
#main function to create table of all assets
def create_universe(onchainfx,assets):
    eligibility = pd.DataFrame(columns = ['Marketcap'])
    eligibility['Marketcap'] = onchainfx['Marketcap']
    eligibility['Current Marketcap'] = onchainfx['Current Marketcap']
    eligibility['Float Percent'] = onchainfx['Supply % Issued']
    eligibility['Float'] = onchainfx['Available Supply']
    eligibility['Price'] = onchainfx['Price']
    eligibility['Total Supply'] = onchainfx['Y2050 Supply']
    eligibility['Volume'] = onchainfx['Est Volume']
    eligibility['Float Pass'] = np.where(eligibility['Float Percent']>=0.1, 1, 0)
    eligibility['Volume Ratio'] = eligibility['Volume'] / eligibility['Current Marketcap']
    eligibility['Volume Pass'] = np.where(eligibility['Volume Ratio']>=volume_cutoff, 1, 0)
    for t in eligibility.index:
        if t in assets.index:
            eligibility.loc[t,('Other Pass')] = assets.loc[t,('Eligible')]
            eligibility.loc[t,('Class')] = assets.loc[t,('Class')]
            eligibility.loc[t,('ERC20')] = assets.loc[t,('ERC20')]
        else:
            print (t,' no other data!')
            eligibility.loc[t,('Other Pass')] = 0
    eligibility['Score'] = eligibility['Volume Pass'] + eligibility['Other Pass'] + eligibility['Float Pass']
    eligibility['Eligible'] = np.where(eligibility['Score']>=2.99999, 1, 0)
    #check if duplicates
    count = len(eligibility.index)
    count1 = len(eligibility.index.unique())
    if count != count1:
        print ('duplicate tickers')
    return eligibility

In [124]:
#rank assets by 2050 marketcap for selection list
def rank_and_sort(df):
    df['Rank'] = df['Marketcap'].rank(ascending=False).astype(int)
    df_sorted = df.sort_values('Rank')
    return df_sorted

In [125]:
#get list of eligible assets and rank them
#really noisy error logging right now due to Messari having 4000 assets
eligibility_db = create_universe(onchainfx_db,asset_db)
eligible_db = eligibility_db[eligibility_db.Eligible != 0]
ineligible_db = eligibility_db[eligibility_db.Eligible == 0]

In [126]:
#make sure I've correctly excluded all of these
double_check = ineligible_db[ineligible_db['Volume Pass'] == 1]
double_check

,Marketcap,Current Marketcap,Float Percent,Float,Price,Total Supply,Volume,Float Pass,Volume Ratio,Volume Pass,Other Pass,Class,ERC20,Score,Eligible
BCN,4.003663e+07,4.003663e+07,1.0,1.844675e+11,0.000217,1.844675e+11,7.853805e+06,1,0.196165,1,0.0,1.0,0.0,2.0,0
CENNZ,6.993961e+07,6.993961e+07,1.0,9.011038e+08,0.077615,9.011038e+08,1.241533e+07,1,0.177515,1,0.0,3.0,0.0,2.0,0
DENT,2.071945e+07,2.071945e+07,1.0,1.000000e+11,0.000207,1.000000e+11,2.347770e+08,1,11.331238,1,0.0,3.0,0.0,2.0,0
DRGN,2.840090e+07,2.840090e+07,1.0,3.392024e+08,0.083728,3.392024e+08,3.076858e+07,1,1.083366,1,0.0,3.0,0.0,2.0,0
ETN,5.668413e+07,5.668413e+07,1.0,1.016806e+10,0.005575,1.016806e+10,6.092356e+07,1,1.074790,1,0.0,3.0,0.0,2.0,0
GRIN,4.322938e+08,4.322938e+08,1.0,9.771011e+08,0.442425,9.771011e+08,1.030483e+09,1,2.383757,1,0.0,1.0,0.0,2.0,0
KCS,6.204210e+07,6.204210e+07,1.0,8.125028e+07,0.763592,8.125028e+07,1.005293e+09,1,16.203405,1,0.0,2.0,0.0,2.0,0
KIN,6.847133e+07,6.847133e+07,1.0,9.995502e+12,0.000007,9.995502e+12,9.525215e+07,1,1.391124,1,0.0,2.0,0.0,2.0,0
PLR,2.032724e+07,2.032724e+07,1.0,8.000000e+08,0.025409,8.000000e+08,2.957972e+06,1,0.145518,1,0.0,3.0,0.0,2.0,0
SMART,4.254080e+06,4.254080e+06,1.0,1.413858e+09,0.003009,1.413858e+09,3.463916e+07,1,8.142575,1,0.0,3.0,0.0,2.0,0


In [127]:
#eligible_db = eligible_db.append(ineligible_db.loc['XTZ'])
#eligible_db = eligible_db.drop('ATOM')

In [128]:
ineligible_db

,Marketcap,Current Marketcap,Float Percent,Float,Price,Total Supply,Volume,Float Pass,Volume Ratio,Volume Pass,Other Pass,Class,ERC20,Score,Eligible
BCN,4.003663e+07,4.003663e+07,1.0,1.844675e+11,0.000217,1.844675e+11,7.853805e+06,1,0.196165,1,0.0,1.0,0.0,2.0,0
CENNZ,6.993961e+07,6.993961e+07,1.0,9.011038e+08,0.077615,9.011038e+08,1.241533e+07,1,0.177515,1,0.0,3.0,0.0,2.0,0
CNX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0.0,3.0,0.0,0.0,0
CTXC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0.0,2.0,0.0,0.0,0
DENT,2.071945e+07,2.071945e+07,1.0,1.000000e+11,0.000207,1.000000e+11,2.347770e+08,1,11.331238,1,0.0,3.0,0.0,2.0,0
DRGN,2.840090e+07,2.840090e+07,1.0,3.392024e+08,0.083728,3.392024e+08,3.076858e+07,1,1.083366,1,0.0,3.0,0.0,2.0,0
EDGE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,1.0,3.0,1.0,1.0,0
ETN,5.668413e+07,5.668413e+07,1.0,1.016806e+10,0.005575,1.016806e+10,6.092356e+07,1,1.074790,1,0.0,3.0,0.0,2.0,0
GBYTE,1.999717e+07,1.999717e+07,1.0,7.511807e+05,26.620980,7.511807e+05,1.249165e+06,1,0.062467,0,1.0,2.0,0.0,2.0,0
GNO,2.618007e+08,2.618007e+08,1.0,1.000000e+07,26.180072,1.000000e+07,2.489434e+07,1,0.095089,0,1.0,3.0,1.0,2.0,0


In [129]:
#create eligible universe - not sure on why there is a warning
universe = rank_and_sort(eligible_db)

C:\Users\stoin\Anaconda2\envs\py35\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [130]:
universe = universe.rename({'HSR' : 'HC'})
universe = universe.rename({'IOT' : 'MIOTA'})
universe = universe.rename({'VEN' : 'VET'})
universe

,Marketcap,Current Marketcap,Float Percent,Float,Price,Total Supply,Volume,Float Pass,Volume Ratio,Volume Pass,Other Pass,Class,ERC20,Score,Eligible,Rank
BTC,1.915570e+11,1.915570e+11,1.0,2.098634e+07,9127.701375,2.098634e+07,2.907861e+12,1,15.180132,1,1.0,1.0,0.0,3.0,1,1
ETH,3.137828e+10,3.137828e+10,1.0,1.351356e+08,232.198398,1.351356e+08,1.127730e+12,1,35.939816,1,1.0,2.0,0.0,3.0,1,2
XRP,1.921466e+10,1.921466e+10,1.0,1.000000e+11,0.192147,1.000000e+11,1.928884e+11,1,10.038605,1,1.0,2.0,0.0,3.0,1,3
LINK,8.485416e+09,8.485416e+09,1.0,1.000000e+09,8.485416,1.000000e+09,1.996054e+11,1,23.523353,1,1.0,3.0,1.0,3.0,1,4
ATOM,8.479005e+09,8.479005e+09,1.0,2.192096e+09,3.867989,2.192096e+09,2.962741e+10,1,3.494209,1,1.0,2.0,0.0,3.0,1,5
XTZ,5.927371e+09,5.927371e+09,1.0,1.923801e+09,3.081073,1.923801e+09,7.059403e+10,1,11.909838,1,1.0,2.0,0.0,3.0,1,6
ADA,5.258241e+09,5.258241e+09,1.0,4.201402e+10,0.125154,4.201402e+10,7.551217e+10,1,14.360730,1,1.0,2.0,0.0,3.0,1,7
XLM,4.716693e+09,4.716693e+09,1.0,5.000000e+10,0.094334,5.000000e+10,7.695940e+10,1,16.316391,1,1.0,1.0,0.0,3.0,1,8
BCH,4.653271e+09,4.653271e+09,1.0,2.098243e+07,221.769868,2.098243e+07,2.006379e+11,1,43.117594,1,1.0,1.0,0.0,3.0,1,9
BSV,3.623686e+09,3.623686e+09,1.0,2.098214e+07,172.703329,2.098214e+07,1.945453e+11,1,53.687143,1,1.0,1.0,0.0,3.0,1,10


In [131]:
#create ten index membership
ten_auto = universe[universe['Rank'] <= 8]
indexes_new_tickers['10'] = pd.DataFrame()
indexes_new_tickers['10'] = ten_auto
ten_band = universe[universe['Rank'] <= 12]
ten_band = ten_band[ten_band['Rank'] > 8]
count_ten_members = len(ten_auto)
for t in ten_band.index:
    if t in indexes_old['10'].index:
        if count_ten_members < 10:
            indexes_new_tickers['10'] = indexes_new_tickers['10'].append(ten_band.loc[t])
            count_ten_members += 1
            print ('Retained' , t)
        else:
            print ('10 full! No room for' , t)
for t in ten_band.index:
    if t not in indexes_old['10'].index:
        if count_ten_members < 10:
            print ('New add' , t)
            indexes_new_tickers['10'] = indexes_new_tickers['10'].append(ten_band.loc[t])
            count_ten_members += 1
        else:
            print ('10 full! Do not add ' , t)

Retained BCH
Retained BSV
10 full! No room for EOS
10 full! Do not add  TRX


In [132]:
#create twenty index membership
twenty_auto = universe[universe['Rank'] <= 26]
for t in twenty_auto.index:
    if t in indexes_new_tickers['10'].index:
        twenty_auto = twenty_auto.drop(t)
#remove all rows in indexes_new_tickers['10']
indexes_new_tickers['20'] = pd.DataFrame()
indexes_new_tickers['20'] = twenty_auto
twenty_band = universe[universe['Rank'] <= 34]
twenty_band = twenty_band[twenty_band['Rank'] > 26]
count_twenty_members = len(twenty_auto)
for t in twenty_band.index:
    if t in indexes_old['20'].index:
        if count_twenty_members < 20:
            indexes_new_tickers['20'] = indexes_new_tickers['20'].append(twenty_band.loc[t])
            count_twenty_members += 1
            print ('Retained' , t)
        else:
            print ('20 full! No room for' , t)
for t in twenty_band.index:
    if t not in indexes_old['20'].index:
        if count_twenty_members < 20:
            print ('New add' , t)
            indexes_new_tickers['20'] = indexes_new_tickers['20'].append(twenty_band.loc[t])
            count_twenty_members += 1
        else:
            print ('20 full! Do not add ' , t)

Retained ZRX
Retained ZIL
Retained BAT
Retained DCR
20 full! Do not add  SC
20 full! Do not add  KNC
20 full! Do not add  WAVES
20 full! Do not add  QTUM


In [133]:
#create forty index membership
forty_auto = universe[universe['Rank'] <= 60]
for t in forty_auto.index:
    if t in indexes_new_tickers['10'].index:
        forty_auto = forty_auto.drop(t)
    if t in indexes_new_tickers['20'].index:
        forty_auto = forty_auto.drop(t)
indexes_new_tickers['40'] = pd.DataFrame()
indexes_new_tickers['40'] = forty_auto
forty_band = universe[universe['Rank'] <= 80]
forty_band = forty_band[forty_band['Rank'] > 60]
count_forty_members = len(forty_auto)
for t in forty_band.index:
    if t in indexes_old['40'].index:
        if count_forty_members < 40:
            indexes_new_tickers['40'] = indexes_new_tickers['40'].append(forty_band.loc[t])
            count_forty_members += 1
            print ('Retained' , t)
        else:
            print ('40 full! No room for' , t)
for t in forty_band.index:
    if t not in indexes_old['40'].index:
        if count_forty_members < 40:
            indexes_new_tickers['40'] = indexes_new_tickers['40'].append(forty_band.loc[t])
            count_forty_members += 1
            print ('New add' , t)
        else:
            print ('40 full! Do not add ' , t)

Retained ARDR
Retained GNT
Retained ELF
Retained GXC
Retained WAN
New add STRAT
New add ANT
New add EDO
New add FUN
New add POLY
40 full! Do not add  POWR
40 full! Do not add  QASH
40 full! Do not add  IOTX
40 full! Do not add  WTC
40 full! Do not add  CVC
40 full! Do not add  LBC
40 full! Do not add  NAS
40 full! Do not add  MTL
40 full! Do not add  SYS
40 full! Do not add  LOOM


In [134]:
#construct total index from 10 + 20 + 40
indexes_new_tickers['Total'] = indexes_new_tickers['10'].append(indexes_new_tickers['20']).append(indexes_new_tickers['40'])

In [135]:
#construct various sector indexes from total index
indexes_new_tickers['ETH'] = indexes_new_tickers['Total'][indexes_new_tickers['Total']['ERC20'] == 1]
indexes_new_tickers['Currency'] = indexes_new_tickers['Total'][indexes_new_tickers['Total']['Class'] == 1]
indexes_new_tickers['Platform'] = indexes_new_tickers['Total'][indexes_new_tickers['Total']['Class'] == 2]
indexes_new_tickers['Application'] = indexes_new_tickers['Total'][indexes_new_tickers['Total']['Class'] == 3]

In [136]:
#print out membership changes for checking and blog
for k in indexes_old:
    print (k , 'adds' , indexes_new_tickers[k].index.difference(indexes_old[k].index).values)
    print (k , 'deletes' , indexes_old[k].index.difference(indexes_new_tickers[k].index).values)
    print (k , 'count members:' , len(indexes_new_tickers[k]))
    if indexes_new_tickers[k].index.duplicated(keep='first').any() == True:
        print ('Duplicates if True' , indexes_new_tickers[k].index.duplicated(keep='first'))
    print ("")
sectors = len(indexes_new_tickers['Currency'])+len(indexes_new_tickers['Platform'])+len(indexes_new_tickers['Application'])
print ('Sector count members:' , sectors)

Currency adds []
Currency deletes ['LTC']
Currency count members: 18

10 adds ['ADA' 'ATOM']
10 deletes ['EOS' 'LTC']
10 count members: 10

Application adds ['ANT' 'BNT' 'EDO' 'FUN' 'POLY' 'WAX']
Application deletes ['LRC' 'MAID' 'MANA' 'MCO' 'RCN' 'REP']
Application count members: 25

20 adds ['EOS']
20 deletes ['ADA']
20 count members: 20

40 adds ['ANT' 'BNT' 'EDO' 'FUN' 'POLY' 'STRAT' 'WAX']
40 deletes ['LRC' 'LSK' 'MAID' 'MANA' 'MCO' 'RCN' 'REP']
40 count members: 40

ETH adds ['ANT' 'BNT' 'FUN' 'POLY' 'WAX']
ETH deletes ['LRC' 'MANA' 'MCO' 'REP']
ETH count members: 21

Total adds ['ANT' 'ATOM' 'BNT' 'EDO' 'FUN' 'POLY' 'STRAT' 'WAX']
Total deletes ['LRC' 'LSK' 'LTC' 'MAID' 'MANA' 'MCO' 'RCN' 'REP']
Total count members: 70

Platform adds ['ATOM' 'STRAT']
Platform deletes ['LSK']
Platform count members: 27

Sector count members: 70


In [137]:
#write off new index memberships for use in rebalance
for k in indexes_new_tickers:
    if 'MIOTA' in indexes_new_tickers[k].index:
        indexes_new_tickers[k] = indexes_new_tickers[k].rename({'MIOTA' : 'IOT'})
    indexes_new_tickers[k].to_csv('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/index-data/src/main/resources/business_rules/'+k+'_pre_'+new_month+'.csv',columns=['Float'])